In [ ]:
from time import time
import cv2
import numpy as np
from keras.models import load_model
from skimage.transform import resize, pyramid_reduce
import PIL
from PIL import Image

model = load_model('islcnnmodel11.h5')
previous = time()
delta = 0

def prediction(pred):
    if(pred>9):
        return (chr(pred+65+1))
    else:
        return (chr(pred+65))


def keras_predict(model, image):
    data = np.asarray( image, dtype="int32" )
    
    pred_probab = model.predict(data)[0]
    pred_class = list(pred_probab).index(max(pred_probab))
    return max(pred_probab), pred_class

def keras_process_image(img):
    
    img = cv2.resize(img, (1,64,64), interpolation = cv2.INTER_AREA)
    return img
 

def crop_image(image, x, y, width, height):
    return image[y:y + height, x:x + width]

def main():
    previous = time()
    delta = 0
    while True:
        current = time()
        delta += current - previous
        previous = current
        if delta > 3:
        # Operations on image
        # Reset the time counter
            delta = 0
        
        cam_capture = cv2.VideoCapture(0)
        _, image_frame = cam_capture.read()  
    # Select ROI
        im2 = crop_image(image_frame, 150,150,250,250)
        image_grayscale = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)

        converted = cv2.cvtColor(im2, cv2.COLOR_BGR2HSV) # Convert from RGB to HSV

        lowerBoundary = np.array([0,40,30],dtype="uint8")
        upperBoundary = np.array([43,255,254],dtype="uint8")

        skinMask = cv2.inRange(converted, lowerBoundary, upperBoundary)
        skinMask = cv2.addWeighted(skinMask,0.5,skinMask,0.5,0.0)
        
        skinMask = cv2.medianBlur(skinMask, 5)
        skin = cv2.bitwise_and(im2, im2, mask = skinMask)

        bw_image = cv2.cvtColor(skin, cv2.COLOR_HSV2BGR)  
        bw_image = cv2.cvtColor(skin, cv2.COLOR_BGR2GRAY)  
        bw_image = cv2.GaussianBlur(bw_image,(5,5),0) 
    
        im3 = cv2.resize(bw_image, (64,64), interpolation = cv2.INTER_AREA)
    
        im4 = np.resize(im3, (64, 64, 3))
        im5 = np.expand_dims(im4, axis=0)
    
        pred_probab, pred_class = keras_predict(model, im5)
    
        curr = prediction(pred_class)
        print(curr)
        cv2.putText(image_frame, curr, (300, 150), cv2.FONT_HERSHEY_COMPLEX, 4.0, (255, 255, 255), lineType=cv2.LINE_AA)
            

    # Display cropped image
        cv2.rectangle(image_frame, (150, 150), (400, 400), (255, 255, 00), 3)
        cv2.imshow("frame",image_frame)
        
        
    #cv2.imshow("Image4",resized_img)
        cv2.imshow("Image3",bw_image)
        cv2.waitKey(1)

        if cv2.waitKey(25) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
                break


if __name__ == '__main__':
    main()

cam_capture.release()
cv2.destroyAllWindows()